In [12]:
# PATH = pathlib.Path(__file__).parent
# DATA_PATH = PATH.joinpath("./datasets").resolve()
# df_all = pd.read_csv(DATA_PATH.joinpath("everything_grouped.csv")).rename(columns={'school_dist':'dist'})
# df_edu_cnty = pd.read_csv(DATA_PATH.joinpath("education_county.csv")).rename(columns={'school_dist':'dist'})

# df_geo_dist = pd.read_csv(DATA_PATH.joinpath('geo_county_school'))[[
#     'county', 'dist', 'geo_county_point', 'geo_dist_point']]

In [13]:
def head(*args, n=3):
    for df in args:
        print("COLUMNS: ", df.shape[1], '\n', 'ROWS:    ', df.shape[0], sep="")
        display(df.head(n))

In [14]:
import sys
import pandas as pd, numpy as np
import geopandas as gp
from shapely import wkt
import folium as fl
import seaborn as sns
import pathlib

from geo_df import GDF
from functions import *

DATA_PATH = "datasets"

In [15]:
df_all = pd.read_csv(f'{DATA_PATH}/everything_grouped_ready.csv')

df_edu_dist = pd.read_csv(f'{DATA_PATH}/education_dist_ready.csv')
df_edu_dist = df_edu_dist[df_edu_dist.pupil_total > 150]

head(df_all, df_edu_dist)

COLUMNS: 355
ROWS:    490


,year,county,geo_county_point,geo_county,cr_count,against_person,against_property,against_society,not_a_crime,cr_rate,...,white_mobile_instances_rate,black_stable_rate,black_mobile_rate,black_mobile_instances_rate,hispanic_stable_rate,hispanic_mobile_rate,hispanic_mobile_instances_rate,asian_stable_rate,asian_mobile_rate,asian_mobile_instances_rate
0,2012,ADAMS,POINT (-104.1930918 39.8398269),MULTIPOLYGON (((-103.70574149517748 39.9999110...,35891,5041.0,23469.0,7381.0,0.0,7794.0,...,32.45,55.0,45.08,47.91,67.0,32.71,36.4,79.0,21.37,23.54
1,2013,ADAMS,POINT (-104.1930918 39.8398269),MULTIPOLYGON (((-103.70574149517748 39.9999110...,33220,4593.0,23863.0,4763.0,1.0,7070.0,...,32.45,55.0,45.08,47.91,67.0,32.71,36.4,79.0,21.37,23.54
2,2014,ADAMS,POINT (-104.1930918 39.8398269),MULTIPOLYGON (((-103.70574149517748 39.9999110...,34494,4975.0,24832.0,4686.0,1.0,7188.0,...,32.45,55.0,45.08,47.91,67.0,32.71,36.4,79.0,21.37,23.54


COLUMNS: 142
ROWS:    155


,county,dist,geo_county_point,geo_dist_point,geo_county,geo_dist,pupil_total,stable,mobile,mobile_instances,...,white_mobile_instances_rate,black_stable_rate,black_mobile_rate,black_mobile_instances_rate,hispanic_stable_rate,hispanic_mobile_rate,hispanic_mobile_instances_rate,asian_stable_rate,asian_mobile_rate,asian_mobile_instances_rate
0,ADAMS,MAPLETON 1,POINT (-104.1930918 39.8398269),POINT (-104.9187196 39.8415103),MULTIPOLYGON (((-103.70574149517748 39.9999110...,MULTIPOLYGON (((-105.01581612299998 39.8144774...,9037,5077,3919,4133,...,52.08,48.04,51.40,53.63,60.31,39.19,42.19,47.22,52.78,53.70
1,ADAMS,ADAMS 12 FIVE STAR SCHOOLS,POINT (-104.1930918 39.8398269),POINT (-104.9668135 39.9262994),MULTIPOLYGON (((-103.70574149517748 39.9999110...,MULTIPOLYGON (((-105.05310614499996 39.9302934...,49889,34283,15424,16854,...,32.27,55.98,43.94,47.41,67.30,32.23,36.81,81.07,18.84,21.15
2,ADAMS,ADAMS COUNTY 14,POINT (-104.1930918 39.8398269),POINT (-104.9268419 39.8059605),MULTIPOLYGON (((-103.70574149517748 39.9999110...,MULTIPOLYGON (((-104.96883410999999 39.7910064...,8265,5510,3038,3397,...,47.45,48.20,51.80,53.60,68.67,35.20,39.57,80.00,20.00,20.00


In [16]:
def add_points(m, df, loc, val, scale=10):
    for geo, v, v_norm in zip(df[loc], df[val], normalize(list(df[val]), scale)):
        fl.CircleMarker(
            location=(geo.y, geo.x),
            radius=v_norm,
            color='white',
            fill=True,
            tooltip=f'{val}: {v}'
        ).add_to(m)
    return m

def add_marks(m, df, loc, var_name, tooltip_title=None):
    if not tooltip_title:
        tooltip_title = var_name
    for geo, name in zip(df[loc], df[var_name]):
        fl.Marker(
            location=(geo.y, geo.x),
            icon=fl.Icon(color="gray", icon="info-sign"),
            opacity=0.75,
            tooltip=f'''{tooltip_title}:<br><b>{name}</b>'''
        ).add_to(m)
    return m

In [17]:
def plot_edu_dist(df, by_1,
        by_2,
        details_1,
        point_scale,
        show_county,
        show_county_marks,
        reverse_cmap,
    ):

    cmap='flare',
    style_kwds=dict(
        fillOpacity=0.5,
        weight=2,
        ),
    highlight_kwds=dict(
        fillOpacity=0.8
        )


    if details_1 == None or details_1 == []:
        details_1 = [by_1]
    
    if reverse_cmap == True:
        cmap = f'{cmap}_r'

    main = GDF(df, geo='geo_dist').df()

    if show_county == True:
        county = GDF(df, geo='geo_county').df()

        county_map = county.explore(style_kwds=dict(fill=False, color='gray', weight=4))

        result = main.explore(tooltip=['dist']+details_1, column=main[by_1],
                    m=county_map, cmap=cmap, vmin=0, vmax=100,
                    style_kwds=style_kwds, highlight_kwds=highlight_kwds)
    else:
        result = main.explore(tooltip=['dist']+details_1, column=main[by_1],
                    cmap=cmap, vmin=0, vmax=100,
                    style_kwds=style_kwds, highlight_kwds=highlight_kwds)
    
    if by_2:
        result = add_points(result, main, 'geo_dist_point', by_2, point_scale)
    return result


by_1 = 'poor_graduated_rate'
by_2 = 'pupil_total'
details_1 = None
reverse_cmap=True
show_dist = False
show_dist_marks = True
point_scale=15

# plot_edu_dist(df_edu_dist, by_1=by_1, by_2=by_2, details_1=['pupil_total'], reverse_cmap=True)

In [21]:
def plot_county(df, by_1, dist, year,
        by_2,
        details_1,
        point_scale,
        show_alt,
        show_alt_marks,
        reverse_cmap,
    ):

    cmap='flare'
    style_kwds=dict(
        fillOpacity=0.5,
        weight=2,
        )
    highlight_kwds=dict(
        fillOpacity=0.8
        )

    df = df[df.year == year]

    if details_1 == None or details_1 == []:
        details_1 = [by_1]

    if reverse_cmap == True:
        cmap = f'{cmap}_r'

    main = GDF(df, geo='geo_county').df()

    color_range = dict()
    if (50 < main[by_1].max() < 101) and (0 < main[by_1].min() < 50):
        color_range = dict(vmin=0, vmax=100)

    if show_alt == True or show_alt_marks == True:
        dist = GDF(dist, geo='geo_dist').df()
    
    if show_alt == True:

        dist_map = dist.explore(style_kwds=dict(fill=False, color='#bababa', weight=4))

        result = main.explore(tooltip=['county']+details_1, column=main[by_1],
                    m=dist_map, cmap=cmap, **color_range,
                    style_kwds=style_kwds, highlight_kwds=highlight_kwds)
    else:
        result = main.explore(tooltip=['county']+details_1, column=main[by_1],
                    cmap=cmap, **color_range,
                    style_kwds=style_kwds, highlight_kwds=highlight_kwds)
    
    if show_alt_marks == True:
        result = add_marks(result, dist, 'geo_dist_point', 'dist', 'School District')
    
    if by_2:
        result = add_points(result, main, 'geo_county_point', by_2, point_scale)

    return result


year = 2012
by_1 = 'pop'
by_2 = 'cr_rate'
details_1 = None
reverse_cmap=True
show_alt = False
show_alt_marks = False
point_scale=15

fig = plot_county(df_all, dist=df_edu_dist, year=year, by_1=by_1, by_2=by_2, details_1=details_1, point_scale=point_scale, show_alt=show_dist, show_alt_marks=show_alt_marks, reverse_cmap=reverse_cmap)
fig.save('map.html')
fig

In [19]:
def plot(type, df, by_1,
        dist:pd.DataFrame   =pd.DataFrame(),
        year:int            =None,
        by_2:str            =None,
        details_1:list      =None,
        point_scale:float   =10,
        show_alt:bool      =False,
        show_alt_marks:bool=False,
        reverse_cmap:bool   =True,
    ):
    if type == 'county':
        if year == None or dist.empty:
            raise ValueError("Must specify year and district edu data if doing county")

        return plot_county(df, by_1, dist, year, by_2, details_1, point_scale, show_alt, show_alt_marks, reverse_cmap)
    
    return plot_edu_dist(df, by_1, by_2, details_1, point_scale, show_alt, show_alt_marks, reverse_cmap)
)

SyntaxError: unmatched ')' (612920025.py, line 18)

In [ ]:
df_all

,year,county,geo_point,geo,cr_count,against_person,against_property,against_society,not_a_crime,cr_rate,...,white_mobile_instances_rate,black_stable_rate,black_mobile_rate,black_mobile_instances_rate,hispanic_stable_rate,hispanic_mobile_rate,hispanic_mobile_instances_rate,asian_stable_rate,asian_mobile_rate,asian_mobile_instances_rate
0,2012,ADAMS,POINT (-104.1930918 39.8398269),MULTIPOLYGON (((-103.70574149517748 39.9999110...,35891,5041.0,23469.0,7381.0,0.0,0.077945,...,32.45,54.66,45.08,47.91,67.49,32.71,36.4,78.55,21.37,23.54
1,2013,ADAMS,POINT (-104.1930918 39.8398269),MULTIPOLYGON (((-103.70574149517748 39.9999110...,33220,4593.0,23863.0,4763.0,1.0,0.070699,...,32.45,54.66,45.08,47.91,67.49,32.71,36.4,78.55,21.37,23.54
2,2014,ADAMS,POINT (-104.1930918 39.8398269),MULTIPOLYGON (((-103.70574149517748 39.9999110...,34494,4975.0,24832.0,4686.0,1.0,0.071877,...,32.45,54.66,45.08,47.91,67.49,32.71,36.4,78.55,21.37,23.54
3,2015,ADAMS,POINT (-104.1930918 39.8398269),MULTIPOLYGON (((-103.70574149517748 39.9999110...,34716,4867.0,24902.0,4947.0,0.0,0.070798,...,32.45,54.66,45.08,47.91,67.49,32.71,36.4,78.55,21.37,23.54
4,2016,ADAMS,POINT (-104.1930918 39.8398269),MULTIPOLYGON (((-103.70574149517748 39.9999110...,61480,8710.0,43928.0,8842.0,0.0,0.123498,...,32.45,54.66,45.08,47.91,67.49,32.71,36.4,78.55,21.37,23.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,2015,YUMA,POINT (-102.2547919 40.1448234),MULTIPOLYGON (((-102.0515511138711 40.43985046...,77,29.0,44.0,4.0,0.0,0.007669,...,23.45,40.00,60.00,80.00,73.95,26.05,31.4,100.00,0.00,0.00
486,2016,YUMA,POINT (-102.2547919 40.1448234),MULTIPOLYGON (((-102.0515511138711 40.43985046...,208,38.0,164.0,6.0,0.0,0.020694,...,23.45,40.00,60.00,80.00,73.95,26.05,31.4,100.00,0.00,0.00
487,2017,YUMA,POINT (-102.2547919 40.1448234),MULTIPOLYGON (((-102.0515511138711 40.43985046...,70,23.0,45.0,2.0,0.0,0.007021,...,23.45,40.00,60.00,80.00,73.95,26.05,31.4,100.00,0.00,0.00
488,2018,YUMA,POINT (-102.2547919 40.1448234),MULTIPOLYGON (((-102.0515511138711 40.43985046...,44,21.0,22.0,1.0,0.0,0.004419,...,23.45,40.00,60.00,80.00,73.95,26.05,31.4,100.00,0.00,0.00


In [ ]:
import pyproj
import plotly.express as px

df = main.copy()

df = df.df()

df.to_crs(pyproj.CRS.from_epsg(4326), inplace=True)

fig = px.choropleth(
        df, geojson=df.geometry, locations=df.index, color='pupil_total'
    ).update_geos(
        fitbounds="locations", visible=False
    ).update_layout(
        margin={'r':10, 't':0, 'l':0, 'b':0}
    )
fig.show()
# df.explore()
# df.plot(figsize=(20, 10))